# Aina Finetune Pipeline
***

QLoRA 4bits small model openthaigpt/openthaigpt-1.0.0-7b-chat

# Install Libraries

In [5]:
#!pip install -q accelerate peft bitsandbytes transformers trl datasets einops sentencepiece

# Import Libraries

In [6]:
import os
import torch
from datasets import load_dataset, Dataset,DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig
import sentencepiece
import pandas as pd
from sklearn.model_selection import train_test_split

# Choosing Model

In [7]:
# Modle from huggingface
model_name = "openthaigpt/openthaigpt-1.0.0-7b-chat"

# Finetuned model name
new_model = "openthaigpt/openthaigpt-1.0.0-7b-chat-customed"

# QLoRA Parameters

In [8]:
lora_r = 8
lora_alpha = 16
lora_dropout = 0.1

# BitsAndBytes Parameters

In [9]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Training Parameters

In [10]:
#  Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

#  Number of training epochs
num_train_epochs = 1

#  Enable fp16/bf16 (set bf16 to True with an A100)

fp16 = False
bf16 = False

#  Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4 

#  Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

#  Enable gradient checkpointing
gradient_checkpointing = True

#  Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

#  Initial learning rate (AdamW optimizer)
learning_rate = 1e-5

#  Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

#  Optimizer to use
optim = "paged_adamw_32bit" # ["paged_adamw_32bit", "adamw_8bit"]

#  Learning rate schedule
lr_scheduler_type = "cosine" # ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]

#  Number of training steps (overrides num_train_epochs)
max_steps = -1

#  Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.1

#  Group sequences into batches with same length Saves memory and speeds up training considerably
group_by_length = True

#  Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# SFT Parameters

In [11]:
#  Maximum sequence length to use
max_seq_length = None

#  Pack multiple short examples in the same input sequence to increase efficiency
packing = False

#  Load the entire model on the GPU 0
device_map = {"": 0}

# Datasets

In [12]:
dataset_name = "Thaweewat/chatmed-5k-th"
dataset = load_dataset(dataset_name)
print("Details of the dataset")
print(dataset)

Generating train split: 100%|██████████| 5452/5452 [00:00<00:00, 88284.43 examples/s]

Details of the dataset
DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input'],
        num_rows: 5452
    })
})


In [13]:
dataset = dataset.map(lambda example: {
    'text': f"<s>[INST] {example['instruction']} [INST] {example['output']} </s>",
})

print("Training data")
print(dataset)

Map: 100%|██████████| 5452/5452 [00:00<00:00, 20196.73 examples/s]

Training data
DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'input', 'text'],
        num_rows: 5452
    })
})


# Train

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_compute_dtype=getattr(torch, bnb_4bit_compute_dtype),
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    use_double_quant=use_nested_quant,
)

# Load base model in 4-bit mode
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

# Load Llama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
perf_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set sft parameters
sft_arguments = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_checkpointing=gradient_checkpointing,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    dataset_text_field="text",
    report_to="tensorboard",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=sft_arguments,
    peft_config=perf_config,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained_model(new_model)

PackageNotFoundError: No package metadata was found for bitsandbytes